# Conversational Voice AI with Knowledge Representation of Extended Warehouse Management

# Backend Technologies:  
   1. Azure -Cognitive AI      for NLP Services
   2. SAP S4 Embedded EWM 9.3   for Warehouse Operations.
   3. SAP ABAP                  for Server Side Validation.
   4. Python 3.3               for Client Side Validations.
   
### Integration Technologies:
   1. Azure Cognitive AI 
   2. Azure NLP (Text and Speech)
   3. SAP Restful API's. 
   4. WEB Hooks
                    
### Frontend Technologies:
   1. UI5 / Adaptive Cards/ Teams Channel
   2. Django / CSS / JS

In [ ]:
# Import namespaces
from dotenv import load_dotenv
from datetime import datetime
import json
import os
import azure.cognitiveservices.speech as speech_sdk
import requests
#Global Keys: Get keys for Azure Cognitive AI
from azure.core.credentials import AzureKeyCredential
from azure.ai.language.questionanswering import QuestionAnsweringClient

In [ ]:
endpoint = "https://sanaai.cognitiveservices.azure.com/"
credential = AzureKeyCredential("d46d00379ac748419ab6c6c346699130")
knowledge_base_project = "SANA-QA"
deployment = "australiaeast"
GV_COGKEY = os.getenv('COG_SERVICE_KEY')
GV_COGREG = os.getenv('COG_SERVICE_REGION')
lv_webhook_url = os.getenv('GV_Webhook')

In [ ]:
# Class to Get Speech Config Parameters by passing keys.
class GCL_S2T:
    # Define Method: Configure speech service
    # Pass the Global keys to Speech SDK and get Speech Config variables.
     
    def config_speech(self): 
        global speech_config
        speech_config = speech_sdk.SpeechConfig(GV_COGKEY, GV_COGREG)
        print('Ready to use speech service in:', speech_config.region)

    # Define Method Transcribe Command

    def TranscribeCommand(self):
        print(speech_config)
     # automatic language detection.
        auto_detect_source_language_config = speech_sdk.languageconfig.AutoDetectSourceLanguageConfig()
    # Configure speech recognition
        audio_config = speech_sdk.AudioConfig(use_default_microphone=True)
        speech_recognizer = speech_sdk.SpeechRecognizer(speech_config, audio_config)  
        print('Listening...........')

    # Process speech input
        speech = speech_recognizer.recognize_once_async().get()
        if speech.reason == speech_sdk.ResultReason.RecognizedSpeech:
             command = speech.text
        else:
            print(speech.reason)

    # Return the command
        return command

In [ ]:
# Class to Read the Knowledge Representations from Language Studio built.
class GCL_QA:
    def get_answer(self, gv_question):
        client = QuestionAnsweringClient(endpoint, credential)
        with client:
            question= gv_question
            output = client.get_answers(
            question = question,
            project_name=knowledge_base_project,
            deployment_name=deployment
            )
        return output

In [ ]:
#language to Convert the Text to Speech
class GCL_T2S:
    def detection_to_speaker(self,text):    
    # create the auto detection language configuration without specific languages
        auto_detect_source_language_config = speech_sdk.languageconfig.AutoDetectSourceLanguageConfig()
        speech_synthesizer = speech_sdk.SpeechSynthesizer( speech_config=speech_config, auto_detect_source_language_config=auto_detect_source_language_config)
        result = speech_synthesizer.speak_text_async(text).get()        
        # Check result

        if result.reason == speech_sdk.ResultReason.SynthesizingAudioCompleted:
            print("Speech synthesized to speaker for text [{}]".format(text))
            stream = speech_sdk.AudioDataStream(result)
            stream.save_to_wav_file(r"output.wav")

In [ ]:
# Initialize objects for all three services (Without Prompt)
obj_t2s = GCL_T2S()
obj_qa  = GCL_QA()
obj_s2t = GCL_S2T()
# Prompt for Question
obj_t2s.detection_to_speaker("This is Sana, your voice AI ! What can i do for you? ")
# Capture the Voice
obj_s2t.config_speech() # Read Method config_speech
gv_question = obj_s2t.TranscribeCommand()
print(gv_question)

# Read the Knowledge Repository 
gv_output = obj_qa.get_answer(gv_question)
gv_answer = gv_output.answers[0].answer
gv_confedence = int(gv_output.answers[0].confidence * 100)

# Print("Accuracy of ", gv_confedence)
# Convert to Speech
          
if gv_confedence > 30:
  read_text = gv_answer  # Send Answer 
  gv_confedence = str(gv_confedence)
  lv_text =  " with a certainity of " + gv_confedence + '%' 
  obj_t2s.detection_to_speaker(lv_text)
  obj_t2s.detection_to_speaker(read_text)
else:
  obj_t2s.detection_to_speaker("I am not so Sure about it! ")